In [554]:
#import Files 
import xml.sax
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords 
import bz2
from nltk.stem.porter import *
import sys
from nltk.stem.snowball import SnowballStemmer



[nltk_data] Downloading package stopwords to /home/apurvi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [555]:
#global Variables 
Doc_ID=1 #to maintain total Number of pages in the dump
Doc_IDtoTitle={} # a map to maintain name of doc ids 

In [556]:
class TitleProcessor():
    def processData(self,data):
#         data=self.camelCase(data)
#         print(data)
        data=self.tokenizer(data)
        data=self.case_fold(data)
        data=self.removeStopwords(data)
        data=self.doStemming(data)
        return data
        
#     def camelCase(self,data):
#         return re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))',data)
    
    def case_fold(self,data):
        case_Folded= [w.casefold() for w in data]
        return case_Folded
    
    def tokenizer(self,data):
        tokenizedata=re.findall("\d+|[\w]+",data)
        tokenizedata=[key for key in tokenizedata]
        return tokenizedata
    
    def removeStopwords(self,data):
        stop_word = set(stopwords.words('english'))
        removedStopW=[w for w in data if w not in stop_word] 
        return removedStopW
    
    def doStemming(self,data):
        stemmer=SnowballStemmer("english")
        stemmed_data=[]
        for words in data:
            stemmed_data.append(stemmer.stem(words))
        return stemmed_data

In [557]:
class TextFieldProcessor():
    data=None
    def process(self,data):
        self.data=data
        self.data=self.data.casefold()
        self.remove_Regx()
        External_Links=self.get_external_links()
        info,bodytext,category=self.get_InfoBox_category()
        references=self.get_references()
        print("info",info)
        print("bodytext",bodytext)
        print("category",category)
        print("reference",references)
        return self.data
    
    def remove_Regx(self):
        removReg1=re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',re.DOTALL).sub('',self.data)
        self.data=removReg1
        
        removReg2=re.compile(r'{\|(.*?)\|}',re.DOTALL).sub('',self.data)
        self.data=removReg2
        
        removReg3=re.compile(r'{{v?cite(.*?)}}',re.DOTALL).sub('',self.data)
        self.data=removReg3
        
        removReg4=re.compile(r'[.,;_()"/\']',re.DOTALL).sub(' ',self.data)
        self.data=removReg4
        
        removReg5=re.compile(r'\[\[file:(.*?)\]\]',re.DOTALL).sub('',self.data)
        self.data=removReg5
        
        removReg6=re.compile(r'<(.*?)>',re.DOTALL).sub('',self.data)
        self.data=removReg6
        self.data=self.data.replace('_',' ').replace(',','')
        
    def get_Refreg(self):
        Ref_regx=re.compile(r'(==\s?references\s?==(.*?)}}\n\n)', re.DOTALL)
        return Ref_regx
        
    def get_external_links(self):
        external_links=[]
        lines = self.data.split("==external links==")
        no_line=len(lines)
        link_present=0
        if(no_line<=1):
            return external_links
#         print(no_line)
        link_line=lines[1].split('\n')
        link_present=1
        
        for line in link_line:
            if link_present==1 or "*[" in line or "* [" in line :
                external_links.extend(line.split(' '))
            else:
                break
        return external_links
                
    def check_bodyend(self,line):
        return "[[category:" in line or "== external links ==" in line
    
    def get_InfoBox_category(self):
        info=[]
        Text_body=[]
        Category=[]
        flag=0
        lines=self.data.split('\n')
        no_line=len(lines)
        i=0
        Body_flag=1
#         
        while(i<no_line):
            if '{{infobox' in lines[i]:
                count_open=0
                split_1=lines[i].split('{{infobox')
                if(len(split_1)==2):
                    info.extend(split_1[1:])
                while True:
                    if '{{' in lines[i]:
                        openb=lines[i].count('{{')
                        flag=1
                        count_open+=openb
                    
                    if '}}' in lines[i]:
                        close=lines[i].count('}}')
                        flag=0
                        count_open-=close
                    
                    if count_open<=0:
                        break
                    i=i+1
                    
                    if(i<len(lines)) and count_open>0:
                        append_data=lines[i]
                        info.append(append_data)
                    else:
                        break
            
            elif (Body_flag==1):
                #at start of a category or a external link body finishes
                if (self.check_bodyend(lines[i])):
                    Body_flag=0
                Text_body.append(lines[i])
            
            elif "[[category:" in lines[i]:
                category_split=lines[i].split("[[category:")
                l=len(category_split)
                if(l>1):
                    Category.extend(category_split[1:-1])
                    l=len(category_split)
                    Category.append(category_split[-1].split(']]')[0])
            
#                 links=1  #handled above
            
#             elif Ref_regx.search(lines[i]):
#                 reference_signal=1 #handle later
            
#             else:
#                 Text_body.append(lines[i])
            i+=1
        
        return info,Text_body,Category
    
    def process_ref(self,Sref):
        self.data=self.data.replace(Sref.group(0), ' ')
        self.data=self.data.replace(Sref.group(2), ' ')
        return Sref.group(2)

    def get_references(self):
        
                
        

In [558]:
def check():
    global Doc_IDtoTitle
    for key in Doc_IDtoTitle:
        print(key," ",Doc_IDtoTitle[key])

In [559]:
#content Handler code for XML parser
def storeValue(key,value,Dict):
    global Doc_IDtoTitle
    Doc_IDtoTitle[key]=value
    
class parseXML(xml.sax.ContentHandler):
    
    global Doc_ID,Doc_IDtoTitle 
    def __init__(self):
        xml.sax.ContentHandler.__init__(self)
        self.current_tag=""
        self.title_content=""
        self.text_content=""
    
    def startElement(self,tag,attributes):
        self.current_tag=tag
        if tag=='page':
            self.title_content=""
            self.text_content=""

    def characters(self,content):
        if self.current_tag=="title":
            self.title_content+=content
        elif self.current_tag=="text":
            self.text_content+=content
            
    def endElement(self,tag):
        if tag=='page':
            global Doc_ID
            print("----------------------end doc",Doc_ID,"----------------------------")
            if Doc_ID==2:
#                 check()
                sys.exit()
            Doc_ID+=1
        
        elif tag=='title':
#             print(self.title_content)
            storeValue(Doc_ID,self.title_content,Doc_IDtoTitle) 
        
        elif tag=='text':
            #process title
            pr=TitleProcessor()
            processedTitle=pr.processData(self.title_content)
            #print("Processed title : ",processedTitle)
            
            #get fields from text_content
            pt=TextFieldProcessor()
            pt.process(self.text_content)
        self.current_tag=""

In [560]:
if __name__=="__main__":
    Wiki_input_file="./Data/input.bz2"
    source = bz2.BZ2File(Wiki_input_file, "rb")

    #Create a parser Object
    parser= xml.sax.make_parser()

    #turnoff namespaces
    parser.setFeature(xml.sax.handler.feature_namespaces, 0)
    
    #callHandler
    handler= parseXML()
    
    #parse data 
    parser.setContentHandler(handler)
    parser.parse(source)
    

info []
bodytext ['#redirect [[computer accessibility]]', '', '{{r from move}}', '{{r from camelcase}}', '{{r unprintworthy}}']
category []
reference 
----------------------end doc 1 ----------------------------
info []
bodytext ['{{short description|political philosophy and movement}}', '{{redirect2|anarchist|anarchists|other uses|anarchists  disambiguation }}', '{{pp-move-indef}}', '{{good article}}', '{{use dmy dates|date=march 2020}}', '{{use british english|date=january 2014}}', '{{anarchism sidebar}}', '{{basic forms of government}}', '   anarchism    is a [[political philosophy]] and [[political movement|movement]] that rejects all involuntary  coercive forms of [[hierarchy]]  it calls for the abolition of the [[state  polity |state]] which it holds to be undesirable  unnecessary  and harmful ', '', 'the [[history of anarchism]] goes back to [[prehistory]]  when some humans lived in anarchistic societies long before the establishment of formal states  [[realm]]s or [[empire]]s  

SystemExit: 

/home/apurvi/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
